原矩阵
[[5,3],
[4,1]]

In [23]:
# 简化版的 Matrix Completion
import numpy as np

class SimpleMatrixCompletion:
    def __init__(self, matrix_shape, rank, lambda_reg):
        self.matrix_shape = matrix_shape
        self.rank = rank
        self.lambda_reg = lambda_reg
        self.P = np.random.rand(matrix_shape[0], rank)  # 初始化 P 矩阵
        self.Q = np.random.rand(matrix_shape[1], rank)  # 初始化 Q 矩阵

    def train(self, observed_entries, max_iterations=10, tolerance=1e-4):
        for iteration in range(max_iterations):
            # 更新 P 矩阵
            for i in range(self.matrix_shape[0]):
                self.P[i, :] = self._update_vector(self.Q, observed_entries, i, axis=0)
            # 更新 Q 矩阵
            for j in range(self.matrix_shape[1]):
                self.Q[j, :] = self._update_vector(self.P, observed_entries, j, axis=1)

        return self.P @ self.Q.T

    def _update_vector(self, fixed_matrix, observed_entries, index, axis):
        if axis == 0:  # 更新 P[i, :]
            indices = [(index, j) for j in range(self.matrix_shape[1]) if (index, j) in observed_entries]
        else:  # 更新 Q[j, :]
            indices = [(i, index) for i in range(self.matrix_shape[0]) if (i, index) in observed_entries]

        if not indices:  # 如果没有观测值，返回零向量
            return np.zeros(self.rank)

        A = np.array([fixed_matrix[j, :] if axis == 0 else fixed_matrix[i, :] for i, j in indices])
        b = np.array([observed_entries[(i, j)] for i, j in indices])

        return np.linalg.solve(A.T @ A + self.lambda_reg * np.eye(self.rank), A.T @ b)


In [27]:
# 初始化矩阵补全
matrix_shape = (2, 2)
rank = 2
lambda_reg = 0.1
matrix_completion = SimpleMatrixCompletion(matrix_shape, rank, lambda_reg)

# 初始没有任何观测值
observed_entries = {}


In [28]:
# 更新观测值
observed_entries[(0, 0)] = 5

# 训练模型并查看结果
estimated_matrix = matrix_completion.train(observed_entries, max_iterations=10)
print("Step 1 - Estimated Matrix:\n", estimated_matrix)


Step 1 - Estimated Matrix:
 [[4.95046492 0.        ]
 [0.         0.        ]]


In [29]:
# 更新观测值
observed_entries[(0, 1)] = 3

# 训练模型并查看结果
estimated_matrix = matrix_completion.train(observed_entries, max_iterations=10)
print("Step 2 - Estimated Matrix:\n", estimated_matrix)


Step 2 - Estimated Matrix:
 [[4.93369823 2.96021894]
 [0.         0.        ]]


In [30]:
# 更新观测值
observed_entries[(1, 0)] = 4

# 训练模型并查看结果
estimated_matrix = matrix_completion.train(observed_entries, max_iterations=10)
print("Step 3 - Estimated Matrix:\n", estimated_matrix)


Step 3 - Estimated Matrix:
 [[4.96586163 2.95041103]
 [3.93779711 2.33959802]]


In [31]:
# 更新观测值
observed_entries[(1, 1)] = 1

# 训练模型并查看结果
estimated_matrix = matrix_completion.train(observed_entries, max_iterations=10)
print("Step 4 - Estimated Matrix:\n", estimated_matrix)

Step 4 - Estimated Matrix:
 [[4.96047629 2.91340634]
 [3.91643715 1.05770858]]
